In [2]:
!CXXFLAGS+=-stdlib=libc++ maturin develop --release

🔗 Found pyo3 bindings
🐍 Found CPython 3.11 at /Users/josh/josh/projects/tech/riversim/.venv/bin/python
💻 Using `MACOSX_DEPLOYMENT_TARGET=10.7` for x86_64-apple-darwin by default
   Compiling riversim v0.1.0 (/Users/josh/josh/projects/tech/riversim)
   --> src/physics.rs:144:17
    |
144 |             let old_h = self
    |                 ^^^^^ help: if this is intentional, prefix it with an underscore: `_old_h`
    |
    = note: `#[warn(unused_variables)]` on by default

   --> src/physics.rs:155:17
    |
155 |             for cell_index in self
    |                 ^^^^^^^^^^ help: if this is intentional, prefix it with an underscore: `_cell_index`

    Finished release [optimized] target(s) in 10.13s
📦 Built wheel for CPython 3.11 to /var/folders/w9/c8qym84j2pg4fjvx_bzr84bc0000gn/T/.tmpgxdZbR/riversim-0.1.0-cp311-cp311-macosx_10_7_x86_64.whl
🛠 Installed riversim-0.1.0


In [2]:
import contextplot
import matplotlib
import matplotlib.collections
import matplotlib.patches
import numpy as np
import tqdm

import riversim

contextplot.set_defaults()

example = 'advection_1d'
# example = 'singularity_1d'

num_cells = 30
solver = getattr(riversim, example)(num_cells, 10)

dt = 0.001

with contextplot.context_video(f'{example}.mp4', 1, size_inches=(8, 3), frame_rate_hz=20) as cv:
    bar = tqdm.tqdm(range(2000))
    for step in bar:
        if step % 20 == 0:
            with cv.next_frame() as cp:
                z_lattice = solver.z_lattice
                x_vertices = solver.grid.x_axis.vertices
                x_centers = solver.grid.x_axis.centers
                pressure = solver.pressure
                velocity = solver.velocity

                poly_verts = []
                pressures = []
                for i in range(z_lattice.shape[0] - 1):
                    for j in range(z_lattice.shape[2] - 1):
                        poly_verts.append([
                            [x_vertices[i], z_lattice[i, 0, j]],
                            [x_vertices[i + 1], z_lattice[i + 1, 0, j]],
                            [x_vertices[i + 1], z_lattice[i + 1, 0, j + 1]],
                            [x_vertices[i], z_lattice[i, 0, j + 1]],
                        ])
                        pressures.append(pressure[i, 0, j, 1])
                poly_collection = matplotlib.collections.PolyCollection(
                    poly_verts, closed=True, array=pressures, cmap=matplotlib.cm.viridis, edgecolors='black', linewidth=0.2, snap=True)
                outs = cp.ax_raw.add_collection(poly_collection)
                _ = cp.figure.colorbar(outs)

                cp.ax.quiver(
                    np.broadcast_to(x_centers[:, np.newaxis], z_lattice[:-1, 0, :-1].shape),
                    0.25 * (z_lattice[:-1, 0, :-1] + z_lattice[1:, 0, :-1] + z_lattice[:-1, 0, 1:] + z_lattice[1:, 0, 1:]),
                    velocity[:, 0, :, 0, 0],
                    velocity[:, 0, :, 0, 2],
                    pivot='tail',
                    scale=1,
                    scale_units='inches',
                )

                cp.ax.set(xlim=[solver.grid.x_axis.vertices[0], solver.grid.x_axis.vertices[-1]], ylim=[0, 1.5])
                _ = bar.set_description(f't: {step * dt:.4f}, Courant margin: {solver.courant_dt / dt:.4f}', 50)

        try:
            solver.step(dt)
        except:
            break

  0%|          | 0/2000 [00:00<?, ?it/s]

t: 1.9800, Courant margin: 58.1548: 100%|██████████| 2000/2000 [00:28<00:00, 70.02it/s]      


In [2]:
(pressure[5, 0, 1, 1] - pressure[5, 0, 0, 1]) / (z_lattice[5, 0, 1] - z_lattice[5, 0, 0])

-2539.701243732179